In [1]:
import pandas as pd
import re
pd.set_option('display.max_rows', None)  

**Tweets Cleaning Part:**

In [2]:
file_path = 'Fiviz_Tweet_Output.csv'

# Preprocessing the CSV file to remove commas after the second comma in each line
cleaned_lines = []
with open(file_path, 'r', encoding='ISO-8859-1') as file:
    for line in file:
        # Splitting the line by commas
        parts = line.split(',')
        
        if len(parts) > 2:
            # Rejoining the line by keeping the first two parts as it is and removing commas from the rest
            cleaned_line = ','.join(parts[:2]) + ',' + ''.join(parts[2:])
        else:
            cleaned_line = line  # Keeping the line as it is if it has two or fewer parts

        cleaned_lines.append(cleaned_line)

# Writing cleaned data to a temporary file
cleaned_file_path = 'cleaned_Fiviz_Tweet_Output.csv'
with open(cleaned_file_path, 'w', encoding='utf-8') as file:
    file.writelines(cleaned_lines)

# Reading the cleaned file
df_tweets = pd.read_csv(cleaned_file_path, quotechar='"', sep=',', encoding='ISO-8859-1', engine='python')

# Displaying the first few rows to check the result
df_tweets.head(6)

,username,time,body
0,shortvolumes,2023-08-06 04:41:00,Short sale volume (not short interest) for $DB...
1,NewAge69,2023-08-07 13:57:00,$DB feels good to stay on higher levels
2,macroaxis,2023-08-08 09:30:00,$HAIA - Is Healthcare better investment than D...
3,STCKPRO,2023-08-08 10:24:00,$DB NEW ARTICLE : Deutsche Bank: It Is Now Tim...
4,stockilluminati,2023-08-08 10:25:00,$DB stockilluminati.com/db/news... - Deutsche ...
5,Atlas44,2023-08-08 12:26:00,$DB Fucking communists in Italy. Don't they u...


In [3]:
df_tweets.tail()

,username,time,body
142,StockAutomatePro,2023-10-26 07:02:00,$DB Stock is looking strong and set to rise ba...
143,HarveyToronto,2023-10-26 01:51:00,$DB book value per share $35
144,STCKPRO,2023-10-27 01:46:00,$DB NEW ARTICLE : Analysts Are Bullish on Top...
145,STCKPRO,2023-10-28 14:23:00,$DB NEW ARTICLE : Kinsale Capital drops WTW c...
146,BigBreakingWire,2023-10-30 07:49:00,Big Breaking ÃÂÃÂÃÂÃÂ°ÃÂÃÂ ÃÂÃÂ¸...


In [4]:
#Function to clean up the body colum with regex 
def clean_body(text):
    if pd.isnull(text):
        return text 
    
    # 1. delete non-ASCII symbols
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # 2. delete ticker symbols (i.e. $DB, $AAPL)
    text = re.sub(r'\$[A-Za-z0-9]+', '', text)
    
    # 3. delete general symbols like: :, -, ?, !
    text = re.sub(r'[:\-?!]', '', text)
    
    # 4. delete double white spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [5]:
# applying clean_body() function
df_tweets['body_cleaned'] = df_tweets['body'].apply(clean_body)

In [6]:
# comparing messy and cleaned body column
df_tweets[['body', 'body_cleaned']].head(15)

,body,body_cleaned
0,Short sale volume (not short interest) for $DB...,Short sale volume (not short interest) for on ...
1,$DB feels good to stay on higher levels,feels good to stay on higher levels
2,$HAIA - Is Healthcare better investment than D...,Is Healthcare better investment than Deutsche ...
3,$DB NEW ARTICLE : Deutsche Bank: It Is Now Tim...,NEW ARTICLE Deutsche Bank It Is Now Time For S...
4,$DB stockilluminati.com/db/news... - Deutsche ...,stockilluminati.com/db/news... Deutsche Bank I...
5,$DB Fucking communists in Italy. Don't they u...,Fucking communists in Italy. Don't they unders...
6,$DB NEW ARTICLE : Deutsche Bank's Chadha Says ...,NEW ARTICLE Deutsche Bank's Chadha Says 3%5% P...
7,$AGNC - AGNC Investment vs. Deutsche Bank AG (...,AGNC Investment vs. Deutsche Bank AG ( ). Next...
8,Deutsche Bank AG ( $DB ) current chance of dis...,Deutsche Bank AG ( ) current chance of distres...
9,$DB $ANTTY Deutsche Bank Appointed as Deposita...,Deutsche Bank Appointed as Depositary Bank for...


In [7]:
df_tweets.size

588

In [8]:
# removing duplicate rows in the body column if the same user posted it (spam)
df_tweets = df_tweets.drop_duplicates(subset=['username','body_cleaned'])

# reset index
df_tweets = df_tweets.reset_index(drop=True)

df_tweets = df_tweets.drop(columns=['body'])

In [9]:
df_tweets.size

399

**Save cleaned Tweets Dataset:**

In [10]:
df_tweets.to_csv('df_tweets_cleaned.csv', index=False)

**KPI Cleaning Part:**

In [11]:
df_kpi = pd.DataFrame()
df_kpi = pd.read_csv('Fiviz_KPI_Output.csv', sep=',', quotechar='"', engine='python')

In [12]:
df_kpi.head()

,Date,Shs Outstand,Perf Week,Shs Float,Short Interest,Target Price,Rel Volume,Prev Close,Shortable,Avg Volume,Price,Recom,Volume,Change
0,"06-08-2023 12:51:28,2.08B,-0.81%,1.88B,13.29M,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"07-08-2023 18:53:34,2.08B,-0.49%,1.88B,13.29M,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"08-08-2023 19:22:16,2.08B,-0.55%,1.88B,13.29M,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"09-08-2023 21:03:29,2.08B,1.12%,1.88B,13.29M,1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"10-08-2023 21:15:37,2.08B,0.18%,1.88B,13.29M,1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Splitting the Date Column
df_kpi = df_kpi['Date'].str.split(',', expand=True)

df_kpi.columns = ['Date', 'Shs Outstand', 'Perf Week', 'Shs Float', 
                    'Short Interest', 'Target Price', 'Rel Volume', 
                    'Prev Close', 'Shortable', 'Avg Volume', 'Price', 
                    'Recom', 'Volume', 'Change']
df_kpi.head()

,Date,Shs Outstand,Perf Week,Shs Float,Short Interest,Target Price,Rel Volume,Prev Close,Shortable,Avg Volume,Price,Recom,Volume,Change
0,06-08-2023 12:51:28,2.08B,-0.81%,1.88B,13.29M,12.02,0.60,10.96,Yes,2.62M,11.02,4.00,"""1.573.588""",0.55%
1,07-08-2023 18:53:34,2.08B,-0.49%,1.88B,13.29M,12.02,0.47,11.02,Yes,2.52M,11.07,4.00,"""1.602.224""",0.50%
2,08-08-2023 19:22:16,2.08B,-0.55%,1.88B,13.29M,12.02,0.51,09.80,Yes,2.50M,09.72,4.00,"""2.680.200""",0.49%
3,09-08-2023 21:03:29,2.08B,1.12%,1.88B,13.29M,12.02,0.61,10.75,Yes,2.48M,10.85,4.00,"""1.245.519""",0.93%
4,10-08-2023 21:15:37,2.08B,0.18%,1.88B,13.29M,12.02,0.75,10.82,Yes,2.49M,10.98,4.00,"""1.588.257""",1.48%


In [14]:
# Checking data types
df_kpi.dtypes

Date              object
Shs Outstand      object
Perf Week         object
Shs Float         object
Short Interest    object
Target Price      object
Rel Volume        object
Prev Close        object
Shortable         object
Avg Volume        object
Price             object
Recom             object
Volume            object
Change            object
dtype: object

In [15]:
# Removing quotes and periods from the 'Volume' column
df_kpi['Volume'] = df_kpi['Volume'].str.replace('"', '').str.replace('.', '')

# Converting the 'Volume' column to int
df_kpi['Volume'] = df_kpi['Volume'].astype(int)

# Checking data types
df_kpi.dtypes

Date              object
Shs Outstand      object
Perf Week         object
Shs Float         object
Short Interest    object
Target Price      object
Rel Volume        object
Prev Close        object
Shortable         object
Avg Volume        object
Price             object
Recom             object
Volume             int32
Change            object
dtype: object

In [16]:
# Function to remove letters; symbols and convert to numbers
def convert_numeric_values(df_kpi):
    def convert_value(value):
        # Check if the value is a string
        if isinstance(value, str):
            # Remove percentage sign
            value = value.replace('%', '')
            # Check for 'B' indicating billions
            if 'B' in value:
                return float(value.replace('B', '')) * 1e9
            # Check for 'M' indicating millions
            elif 'M' in value:
                return float(value.replace('M', '')) * 1e6
            else:
                return float(value)  # Convert to float if it's a pure number
        return value 

    # Applying the conversion function to all columns except 'Date' and 'Shortable'
    for column in df_kpi.columns:
        if column not in ['Date', 'Shortable']:
            df_kpi[column] = df_kpi[column].apply(convert_value)

    return df_kpi

# Applying the function to the DataFrame
df_kpi = convert_numeric_values(df_kpi)

# Rename 'Perf Week' to 'Perf Week_%' and 'Change' to 'Change_%'
df_kpi.rename(columns={'Perf Week': 'Perf Week_%', 'Change': 'Change_%'}, inplace=True)

# Display the converted DataFrame
df_kpi.head()

,Date,Shs Outstand,Perf Week_%,Shs Float,Short Interest,Target Price,Rel Volume,Prev Close,Shortable,Avg Volume,Price,Recom,Volume,Change_%
0,06-08-2023 12:51:28,2.080000e+09,-0.81,1.880000e+09,13290000.0,12.02,0.60,10.96,Yes,2620000.0,11.02,4.0,1573588,0.55
1,07-08-2023 18:53:34,2.080000e+09,-0.49,1.880000e+09,13290000.0,12.02,0.47,11.02,Yes,2520000.0,11.07,4.0,1602224,0.50
2,08-08-2023 19:22:16,2.080000e+09,-0.55,1.880000e+09,13290000.0,12.02,0.51,9.80,Yes,2500000.0,9.72,4.0,2680200,0.49
3,09-08-2023 21:03:29,2.080000e+09,1.12,1.880000e+09,13290000.0,12.02,0.61,10.75,Yes,2480000.0,10.85,4.0,1245519,0.93
4,10-08-2023 21:15:37,2.080000e+09,0.18,1.880000e+09,13290000.0,12.02,0.75,10.82,Yes,2490000.0,10.98,4.0,1588257,1.48


In [17]:
# Checking data types
df_kpi.dtypes

Date               object
Shs Outstand      float64
Perf Week_%       float64
Shs Float         float64
Short Interest    float64
Target Price      float64
Rel Volume        float64
Prev Close        float64
Shortable          object
Avg Volume        float64
Price             float64
Recom             float64
Volume              int64
Change_%          float64
dtype: object

In [18]:
# Convert 'Date' column to datetime with day-first format
df_kpi['Date'] = pd.to_datetime(df_kpi['Date'], format="%d-%m-%Y %H:%M:%S", dayfirst=True)

# Rename 'Date' column to 'Datetime'
df_kpi.rename(columns={'Date': 'Datetime'}, inplace=True)

# Create a new 'Date' column with only the date part (no time)
df_kpi['Date'] = df_kpi['Datetime'].dt.date

# Extract month, weekday, and year
df_kpi['Month'] = df_kpi['Datetime'].dt.month
df_kpi['Weekday'] = df_kpi['Datetime'].dt.day_name()  # Gets the weekday name
df_kpi['Year'] = df_kpi['Datetime'].dt.year

In [19]:
df_kpi.head()

,Datetime,Shs Outstand,Perf Week_%,Shs Float,Short Interest,Target Price,Rel Volume,Prev Close,Shortable,Avg Volume,Price,Recom,Volume,Change_%,Date,Month,Weekday,Year
0,2023-08-06 12:51:28,2.080000e+09,-0.81,1.880000e+09,13290000.0,12.02,0.60,10.96,Yes,2620000.0,11.02,4.0,1573588,0.55,2023-08-06,8,Sunday,2023
1,2023-08-07 18:53:34,2.080000e+09,-0.49,1.880000e+09,13290000.0,12.02,0.47,11.02,Yes,2520000.0,11.07,4.0,1602224,0.50,2023-08-07,8,Monday,2023
2,2023-08-08 19:22:16,2.080000e+09,-0.55,1.880000e+09,13290000.0,12.02,0.51,9.80,Yes,2500000.0,9.72,4.0,2680200,0.49,2023-08-08,8,Tuesday,2023
3,2023-08-09 21:03:29,2.080000e+09,1.12,1.880000e+09,13290000.0,12.02,0.61,10.75,Yes,2480000.0,10.85,4.0,1245519,0.93,2023-08-09,8,Wednesday,2023
4,2023-08-10 21:15:37,2.080000e+09,0.18,1.880000e+09,13290000.0,12.02,0.75,10.82,Yes,2490000.0,10.98,4.0,1588257,1.48,2023-08-10,8,Thursday,2023


In [20]:
df_kpi.dtypes

Datetime          datetime64[ns]
Shs Outstand             float64
Perf Week_%              float64
Shs Float                float64
Short Interest           float64
Target Price             float64
Rel Volume               float64
Prev Close               float64
Shortable                 object
Avg Volume               float64
Price                    float64
Recom                    float64
Volume                     int64
Change_%                 float64
Date                      object
Month                      int32
Weekday                   object
Year                       int32
dtype: object

In [21]:
# Changing Short interest and Avg Volume columns into int
df_kpi['Short Interest'] = df_kpi['Short Interest'].astype(int)
df_kpi['Avg Volume'] = df_kpi['Avg Volume'].astype(int)

In [22]:
df_kpi.head()

,Datetime,Shs Outstand,Perf Week_%,Shs Float,Short Interest,Target Price,Rel Volume,Prev Close,Shortable,Avg Volume,Price,Recom,Volume,Change_%,Date,Month,Weekday,Year
0,2023-08-06 12:51:28,2.080000e+09,-0.81,1.880000e+09,13290000,12.02,0.60,10.96,Yes,2620000,11.02,4.0,1573588,0.55,2023-08-06,8,Sunday,2023
1,2023-08-07 18:53:34,2.080000e+09,-0.49,1.880000e+09,13290000,12.02,0.47,11.02,Yes,2520000,11.07,4.0,1602224,0.50,2023-08-07,8,Monday,2023
2,2023-08-08 19:22:16,2.080000e+09,-0.55,1.880000e+09,13290000,12.02,0.51,9.80,Yes,2500000,9.72,4.0,2680200,0.49,2023-08-08,8,Tuesday,2023
3,2023-08-09 21:03:29,2.080000e+09,1.12,1.880000e+09,13290000,12.02,0.61,10.75,Yes,2480000,10.85,4.0,1245519,0.93,2023-08-09,8,Wednesday,2023
4,2023-08-10 21:15:37,2.080000e+09,0.18,1.880000e+09,13290000,12.02,0.75,10.82,Yes,2490000,10.98,4.0,1588257,1.48,2023-08-10,8,Thursday,2023


**Save cleaned KPI Dataset:**

In [23]:
df_kpi.to_csv('df_kpi_cleaned.csv', index=False)